#### News Classification usingAG_News Data

"World", "Sports", "Business", "Sci/Tech"

In [1]:
!pip install torchdata

In [3]:
import tensorflow
from tensorflow import keras

In [4]:
import torchtext

ModuleNotFoundError: No module named 'torchtext'

In [5]:
import numpy as np

train_dataset, test_dataset = torchtext.datasets.AG_NEWS()
train_dataset = list(train_dataset)
print(train_dataset[0])

X_train_text, Y_train = [],[]\
for Y, X in train_dataset:
    X_train_text.append(X)
    Y_train.append(Y)
    
print(X_train_text[0])

X_test_text, Y_test = [],[]
for Y, X in test_dataset:
    X_test_text.append(X)
    Y_test.append(Y)

print(X_test_text[0])

unique_classes = list
target_classes = ['World', 'Sports', 'Business', 'Sci/Tech']

## Subtract 1 from the labels to make them zero-indexed
Y_train, Y_test = np.array(Y_train) - 1, np.array(Y_test) - 1

len(X_train_text), len(Y_train)

SyntaxError: invalid syntax (3521007970.py, line 8)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tockenizer =  Tockenizer()
tockenizer.fit_on_texts(X_train_text+X_test_text)

print("Vocabulary size: {}".format(len(tockenizer.word_index)))

max_tokens= 50
## Vectorizing data to keep max_tokens words per sample.
X_train_vect = pad_sequences(tockenizer.texts_to_sequences(X_train_text), maxlen=max_tokens, padding='post', truncating='post', value=0)
X_test_vect = pad_sequences(tockenizer.texts_to_sequences(X_test_text), maxlen=max_tokens, padding='post', truncating='post', value=0)

# Check the shape of the vectorized data
print("Shape of X_train_vect: {}".format(X_train_vect.shape))
print("Shape of X_test_vect: {}".format(X_test_vect.shape))

In [ ]:
## What is word 444
print("Word corresponding to index 444: {}".format(tockenizer.index_word[444]))

## How many times it comes in first text document   
print(X_train_vect[0])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

embed_len = 50

model = Sequential([
    layers.Embedding(input_dim=len(tockenizer.word_index)+1, output_dim=embed_len, input_length=max_tokens),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(target_classes), activation='softmax')
]) 

model.summary() 

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_vect, Y_train, epochs=10, batch_size=128, validation_data=(X_test_vect, Y_test))

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

train_preds = model.predict(X_train_vect)
test_preds = model.predict(X_test_vect)

print("Train Accuracy: {}".format(accuracy_score(Y_train, np.argmax(train_preds, axis=1))))
print("Test Accuracy: {}".format(accuracy_score(Y_test, np.argmax(test_preds, axis=1))))
print("Classification Report:\n {}".format(classification_report(Y_test, np.argmax(test_preds, axis=1), target_names=target_classes)))